In [1]:
import json
import pandas as pd

In [2]:
# 엑셀 파일 경로
file_path = 'data/arts.xlsx'
df = pd.read_excel(file_path)
df = df.drop(columns=['_x0008_작품이미지', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13'])
df["이미지"] = df['번호'].apply(lambda x: str(x)+".jpg")
df = df[0:50]
df.to_csv("data/art.csv", index=False)

In [38]:
df = pd.read_csv("data/art.csv")

In [ ]:
import os
from openai import OpenAI
from pprint import pprint

In [104]:
prompt = """
## Role: 미술 작품 설명 추출기

## Instruction
- 미술 작품에 대한 설명이 주어지면 작가에 대한 설명과 작품에 대한 설명을 분리하고 요약한다.
- 아래 Json 포맷에 맞게 답변을 생성한다.
- 한국어로 생성한다.

## Output format
{"art_description": "$art_description", "author_description":"$author_description"}
"""

for i in range(len(df)):
    row_data = df.loc[i]
    content = df['작품 설명'][i]

    client = OpenAI(
        api_key=os.environ["UPSTAGE_API_KEY"], base_url="https://api.upstage.ai/v1/solar"
    )
    chat_result = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": content},
        ],
    )

    output_dict = json.loads(chat_result.choices[0].message.content)
    df.loc[i, 'art_description'] = output_dict['art_description']
    df.loc[i, 'author_description'] = output_dict['author_description']
    

In [115]:
df.loc[0, 'art_description'] = "〈포도 따는 남자〉는 포도 아래 쭈그린 자세로 앉아있는 인물을 그린 작품으로, 흐릿하게 보이는 먼 공간을 표현했습니다. 포도알 하나가 인물의 손보다 크게 그려져 있어 상상적 풍경에 가까운 모습입니다. 그림 속 포도를 따는 모습은 노동의 현장이기보다는 풍요로운 동산에서 자유롭게 수확하고 있는 느낌의 그림으로 프랑스의 화가 폴 고갱(Paul Gauguin)의 타히티 시절 그림을 연상시키기도 합니다."
df.loc[0, 'author_description'] = "이중섭은 소, 아이들 등을 주요 소재로 고분 벽화와 민화 등 전통적이고 토속적인 것에 영감을 받아 표현주의적인 감각으로 작품을 제작했습니다. 이중섭의 작품에서는 그의 삶을 엿볼 수 있습니다. 동경의 분카학원에서 야마모토 마사코와 연애하던 시기의 엽서화에는 두 사람의 연인관계를 암시하는 환상적이고 초현실주의적인 이미지를 그렸습니다. 한국 전쟁기 제주도 피란시절 작품에는 가족과 행복했던 나날들이 소박하게 표현되었으며, 가족을 일본으로 보낸 후에는 삭막한 풍경화와 전쟁의 은유들이 그려졌습니다. 그는 열악한 경제 상황과 재료 부족에도 끊임없이 새로운 기법과 재료를 실험했는데, 담배를 싼 은지를 활용한 은지화가 대표적인 예입니다. 전쟁이 끝난 후에는 가족을 만나려는 생각에 작품 제작에 몰두하여, 당당하고 힘찬 기세가 화면에 나타납니다. 그러나 곧 경제적 어려움과 정신질환 등에 시달리며 가족과 재회할 수 있으리라는 희망이 사라졌을 때에는 초점을 잃은 흐릿한 풍경들이 애잔하게 펼쳐졌습니다."

In [117]:
df.to_csv("data/arts02.csv", index=False)